<a href="https://colab.research.google.com/github/giambono/divine_semantics/blob/main/notebooks/run_compute_qdrant_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os

# Check if the notebook is running on Google Colab
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

# Clone the repository if running on Colab
if is_colab():
    print("Running on Google Colab. Cloning repository...")
    !git clone https://github.com/giambono/divine_semantics.git
    os.chdir("/content/divine_semantics")
    !pip install -r requirements.txt
else:
    import sys
    sys.path.append("..")
    print(f"Working directory set to: {os.getcwd()}")
    print("Not running on Google Colab.")

Working directory set to: /home/rfflpllcn/IdeaProjects/divine_semantics/notebooks
Not running on Google Colab.


In [4]:
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient

load_dotenv()

qdrant_url = os.getenv("QDRANT_URL")
qdrant_api_key = os.getenv("QDRANT_API_KEY")

qdrant_client = QdrantClient(
    url=qdrant_url,
    api_key=qdrant_api_key,
)

In [6]:
import ast
import numpy as np
import pandas as pd
from src.query import evaluate_query
import src.compute_embeddings as compute_embeddings
from src.db_helper import get_db_connection
from src.utils import load_model

import config

In [7]:
collection_name = "dante_multilingual_e5"

model_key = "multilingual_e5"
model = load_model(model_key)

path = os.path.join(config.ROOT, "data/paraphrased_verses.parquet")
test_queries = pd.read_parquet(path)

N = 1
test_queries_sample = test_queries.sample(n=N)[["transformed_text", "expected_index"]]

# Iterate over the sampled rows and collect the evaluation output
out_collect = []
for _, row in test_queries_sample.iterrows():
    query_text = row["transformed_text"]
    expected_index = row["expected_index"]
    result = evaluate_query(
        qdrant_client,
        collection_name,
        query_text,
        expected_index,
        model,
        author_ids=[1, 2, 3, 4, 5],
        type_ids=1
    )
    out_collect.append([query_text, result])

# Calculate performance based on the count of True results
true_count = sum(flag for _, flag in out_collect)
performance = true_count / len(out_collect)

print(f"True count: {performance * 100:.2f}%")

True count: 100.00%


In [8]:
out_collect_df = pd.DataFrame(out_collect, columns=["query_text", "is_correct"])
out_collect_df

,query_text,is_correct
0,"Without hope, we live in desire”",True


In [9]:
out_collect_df.to_clipboard()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
